# Laboratorio 6
 - BRANDON JAVIER REYES MORALES
 - CARLOS ALBERTO VALLADARES GUERRA
 - JUAN PABLO SOLIS ALBIZUREZJUAN PABLO SOLIS ALBIZUREZ

## Task 1

1. __En un juego de suma cero para dos jugadores__

__¿cómo funciona el algoritmo minimax para determinar la estrategia óptima para cada jugador?__


El algoritmo minimax evalua todas las conclusiones de un juego, seleccionando movimientos para obtener beneficios maximizados y minimizados. Se construye un árbol de decisión, donde cada nodo representa un estado del juego, utilizando una función de utilidad para valorar los estados finales, es decir, las hojas del árbol.

__¿Puede explicarnos el concepto de "valor minimax" y su importancia en este contexto?__

El valor minimax es el resultado óptimo que un jugador puede asegurar, enfrentando el oponente de manera efectiva. Este valor es relevante y confiable para enfrentar oponentes más formidables, garantizando el mejor resultado en las mejores condiciones de un juego competitivo.

2. __Compare y contraste el algoritmo minimax con la poda alfa-beta.__ 
__¿Cómo mejora la poda alfa-beta la eficiencia del algoritmo minimax, particularmente en árboles de caza grandes? Proporcione un ejemplo para ilustrar la diferencia en la complejidad computacional entre la poda minimax y alfa-beta.__

El algoritmo minimax, por sí solo, evalúa todas las ramas del árbol de decisión hasta cierta profundidad. Sin embargo, en árboles de gran tamaño, esta búsqueda exhaustiva se torna ineficiente debido al crecimiento exponencial del número de nodos (con una complejidad de O(b^d), donde b representa el factor de ramificación y d la profundidad del árbol).

La poda alfa-beta emerge como una técnica que optimiza la eficiencia del minimax al eliminar (“podar”) aquellas ramas del árbol que no necesitan ser analizadas, puesto que no afectan la decisión final. Así, la poda alfa-beta logra reducir de manera significativa la cantidad de nodos que deben ser evaluados.

__Ejemplo:__
Si tenemos un árbol con factor de ramificación b=10 y profundidad d=6:

 - Minimax estándar tendría que evaluar aproximadamente 10^(6) = 1,000,000 nodos.
 - Con poda alfa-beta, en el mejor caso, el algoritmo podría reducirse a evaluar aproximadamente 10^(3) nodos (unos 1,000 nodos aproximadamente). Esto significa una reducción drástica en el tiempo computacional y una mejora considerable en la eficiencia.


3. __¿Cuál es el papel de expectiminimax en juegos con incertidumbre, como aquellos que involucran nodos de azar o información oculta?__

El algoritmo Expectiminimax maneja juegos con incertidumbre incorporando nodos probabilísticos en el árbol de juego para considerar resultados aleatorios o información oculta mediante valores esperados.__

__¿En qué se diferencia el expectiminimax del minimax en el manejo de resultados probabilísticos y cuáles son los desafíos clave que aborda?__

Minimax asume decisiones deterministas (peor caso), mientras que Expectiminimax maneja resultados probabilísticos usando valores esperados.

| Aspecto                             | Minimax                                                                 | Expectiminimax                                                                                                                                      |
|-------------------------------------|-------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------|
| **Manejo de resultados probabilísticos** | No maneja incertidumbre; asume decisiones deterministas de los jugadores. | Incorpora nodos de azar en el árbol de decisión, permitiendo el manejo de eventos aleatorios y resultados probabilísticos.                           |
| **Estructura del árbol de decisión** | Alterna entre nodos "Max" y "Min" que representan las decisiones de los jugadores. | Intercala nodos "Max", "Min" y "Chance" (azar), donde los nodos de azar calculan el valor esperado basado en las probabilidades de eventos aleatorios. |
| **Aplicación típica**               | Juegos deterministas con información perfecta, como el ajedrez.          | Juegos que incluyen elementos de azar o información incompleta, como el backgammon o juegos de cartas.                                              |
| **Complejidad computacional**       | Crece exponencialmente con la profundidad del árbol y el factor de ramificación. | Similar a Minimax, pero puede ser mayor debido a la inclusión de nodos de azar y el cálculo de valores esperados.                                    |
| **Desafíos clave**                  | - No maneja adecuadamente la incertidumbre o eventos aleatorios.<br>- Limitado a juegos deterministas. | - Manejo de la incertidumbre y eventos aleatorios.<br>- Cálculo de valores esperados en nodos de azar.<br>- Incremento en la complejidad computacional. |


# Task 2

In [ ]:
import tkinter as tk
import tkinter.messagebox as msg
import numpy as np
import math
import random

# Parámetros generales
FILAS = 6
COLUMNAS = 7
TAM_CELDA = 100
MARGEN_CIRCULO = 5

COLOR_FONDO = "blue"
COLOR_VACIO = "white"
COLOR_JUGADOR = "red" # Jugador humano
COLOR_IA = "yellow" # IA


# Funciones de lógica del juego
def crear_tablero():
    return np.zeros((FILAS, COLUMNAS), dtype=int)

def columna_valida(tablero, col):
    return tablero[FILAS-1, col] == 0

def obtener_fila_libre(tablero, col):
    for f in range(FILAS):
        if tablero[f, col] == 0:
            return f
    return None

def colocar_ficha(tablero, fila, col, ficha):
    tablero[fila, col] = ficha

def hay_ganador(tablero, ficha):
    # Horizontal
    for r in range(FILAS):
        for c in range(COLUMNAS - 3):
            if all(tablero[r, c+i] == ficha for i in range(4)):
                return True
    # Vertical
    for c in range(COLUMNAS):
        for r in range(FILAS - 3):
            if all(tablero[r+i, c] == ficha for i in range(4)):
                return True
    # Diagonal positiva
    for r in range(FILAS - 3):
        for c in range(COLUMNAS - 3):
            if all(tablero[r+i, c+i] == ficha for i in range(4)):
                return True
    # Diagonal negativa
    for r in range(3, FILAS):
        for c in range(COLUMNAS - 3):
            if all(tablero[r-i, c+i] == ficha for i in range(4)):
                return True
    return False

def tablero_lleno(tablero):
    return all(tablero[FILAS-1, c] != 0 for c in range(COLUMNAS))

def evaluar_ventana(ventana, ficha):
    puntuacion = 0
    ficha_oponente = 1 if ficha == 2 else 2

    if ventana.count(ficha) == 4:
        puntuacion += 100
    elif ventana.count(ficha) == 3 and ventana.count(0) == 1:
        puntuacion += 5
    elif ventana.count(ficha) == 2 and ventana.count(0) == 2:
        puntuacion += 2

    # Penaliza si el oponente tiene 3 en línea y hay un espacio
    if ventana.count(ficha_oponente) == 3 and ventana.count(0) == 1:
        puntuacion -= 4

    return puntuacion

def puntuar_tablero(tablero, ficha):
    score = 0
    # Extra por controlar la columna central
    columna_central = [int(i) for i in list(tablero[:, COLUMNAS//2])]
    score += columna_central.count(ficha) * 3

    # Revisa filas
    for r in range(FILAS):
        fila = [int(x) for x in tablero[r, :]]
        for c in range(COLUMNAS - 3):
            ventana = fila[c:c+4]
            score += evaluar_ventana(ventana, ficha)

    # Revisa columnas
    for c in range(COLUMNAS):
        col = [int(x) for x in tablero[:, c]]
        for r in range(FILAS - 3):
            ventana = col[r:r+4]
            score += evaluar_ventana(ventana, ficha)

    # Diagonales positivas
    for r in range(FILAS - 3):
        for c in range(COLUMNAS - 3):
            ventana = [tablero[r+i, c+i] for i in range(4)]
            score += evaluar_ventana(ventana, ficha)

    # Diagonales negativas
    for r in range(3, FILAS):
        for c in range(COLUMNAS - 3):
            ventana = [tablero[r-i, c+i] for i in range(4)]
            score += evaluar_ventana(ventana, ficha)

    return score

def columnas_disponibles(tablero):
    return [c for c in range(COLUMNAS) if columna_valida(tablero, c)]

def minimax(tablero, profundidad, maximizando, ficha, alpha=-math.inf, beta=math.inf, usar_poda=True):
    if profundidad == 0 or hay_ganador(tablero, 1) or hay_ganador(tablero, 2) or tablero_lleno(tablero):
        # Caso terminal
        if hay_ganador(tablero, ficha):
            return (None, math.inf)
        elif hay_ganador(tablero, 1 if ficha == 2 else 2):
            return (None, -math.inf)
        else:
            return (None, puntuar_tablero(tablero, ficha))

    validas = columnas_disponibles(tablero)
    if maximizando:
        valor = -math.inf
        mejor_col = random.choice(validas)
        for col in validas:
            fila_libre = obtener_fila_libre(tablero, col)
            copia = tablero.copy()
            colocar_ficha(copia, fila_libre, col, ficha)
            _, nuevo_valor = minimax(copia, profundidad-1, False, ficha, alpha, beta, usar_poda)
            if nuevo_valor > valor:
                valor = nuevo_valor
                mejor_col = col
            if usar_poda:
                alpha = max(alpha, valor)
                if alpha >= beta:
                    break
        return (mejor_col, valor)
    else:
        valor = math.inf
        mejor_col = random.choice(validas)
        ficha_oponente = 1 if ficha == 2 else 2
        for col in validas:
            fila_libre = obtener_fila_libre(tablero, col)
            copia = tablero.copy()
            colocar_ficha(copia, fila_libre, col, ficha_oponente)
            _, nuevo_valor = minimax(copia, profundidad-1, True, ficha, alpha, beta, usar_poda)
            if nuevo_valor < valor:
                valor = nuevo_valor
                mejor_col = col
            if usar_poda:
                beta = min(beta, valor)
                if alpha >= beta:
                    break
        return (mejor_col, valor)

def movimiento_ia(tablero, profundidad, ficha, usar_poda=True):
    col, _ = minimax(tablero, profundidad, True, ficha, usar_poda=usar_poda)
    if col is None or not columna_valida(tablero, col):
        # Si minimax no devolvió algo válido, elige al azar
        col = random.choice(columnas_disponibles(tablero))
    return col

# Clase para la ventana principal de configuración
class VentanaConfiguracion:
    def __init__(self, root):
        self.root = root
        self.root.title("Configuración de Conecta 4")

        self.modo_juego = tk.StringVar(value="humano_vs_ia") # Por defecto
        self.usar_alphabeta = tk.BooleanVar(value=True) # Por defecto
        self.profundidad = tk.IntVar(value=4) # Por defecto

        # Frame para el modo de juego
        frame_modo = tk.LabelFrame(self.root, text="Modo de juego")
        frame_modo.pack(padx=10, pady=10, fill=tk.X)
        tk.Radiobutton(frame_modo, text="Humano vs IA", variable=self.modo_juego, value="humano_vs_ia").pack(anchor=tk.W)
        tk.Radiobutton(frame_modo, text="IA vs IA", variable=self.modo_juego, value="ia_vs_ia").pack(anchor=tk.W)

        # Frame para la profundidad
        frame_profundidad = tk.LabelFrame(self.root, text="Profundidad de búsqueda")
        frame_profundidad.pack(padx=10, pady=10, fill=tk.X)
        tk.Spinbox(frame_profundidad, from_=1, to=10, textvariable=self.profundidad, width=5).pack(pady=5)

        # Frame para la poda alfa-beta
        frame_alphabeta = tk.LabelFrame(self.root, text="Opciones de eficiencia")
        frame_alphabeta.pack(padx=10, pady=10, fill=tk.X)
        tk.Checkbutton(frame_alphabeta, text="Usar Alpha-Beta Pruning", variable=self.usar_alphabeta).pack(anchor=tk.W)

        # Botón para iniciar el juego
        tk.Button(self.root, text="Iniciar Juego", command=self.iniciar_juego).pack(pady=10)

    def iniciar_juego(self):
        modo = self.modo_juego.get()
        profundidad = self.profundidad.get()
        poda = self.usar_alphabeta.get()

        # Cierra la ventana de configuración
        self.root.destroy()

        # Crear ventana del juego
        nueva_ventana = tk.Tk()
        ConnectFourGUI(nueva_ventana, modo, profundidad, poda)
        nueva_ventana.mainloop()

# Clase para la ventana del tablero
class ConnectFourGUI:
    def __init__(self, root, modo_juego, profundidad, usar_poda):
        self.root = root
        self.root.title("Conecta 4 - Partida")
        self.tablero = crear_tablero()
        self.modo = modo_juego
        self.profundidad = profundidad
        self.usar_poda = usar_poda

        self.juego_terminado = False
        self.turno = 0  # 0 => Jugador 1 (humano o IA1), 1 => Jugador 2 (IA o IA2)

        # Etiqueta de estado en la parte superior
        self.lbl_estado = tk.Label(self.root, text="Iniciando partida...", font=("Arial", 14))
        self.lbl_estado.pack(pady=5)

        # Canvas para dibujar el tablero
        ancho = COLUMNAS * TAM_CELDA
        alto = FILAS * TAM_CELDA
        self.canvas = tk.Canvas(self.root, width=ancho, height=alto, bg=COLOR_FONDO)
        self.canvas.pack()
        self.canvas.bind("<Button-1>", self.click_humano)

        self.actualizar_tablero()
        self.actualizar_estado()

        # Si el modo es IA vs IA y el primer turno corresponde a la IA, iniciamos la jugada IA de inmediato
        if self.modo == "ia_vs_ia" and self.turno == 0:
            self.root.after(500, self.jugada_ia)

    def actualizar_tablero(self):
        self.canvas.delete("all")
        for f in range(FILAS):
            for c in range(COLUMNAS):
                x1 = c * TAM_CELDA
                y1 = (FILAS - 1 - f) * TAM_CELDA
                x2 = x1 + TAM_CELDA
                y2 = y1 + TAM_CELDA

                valor = self.tablero[f, c]
                if valor == 0:
                    color = COLOR_VACIO
                elif valor == 1:
                    color = COLOR_JUGADOR
                else:
                    color = COLOR_IA

                self.canvas.create_rectangle(x1, y1, x2, y2, fill=COLOR_FONDO, outline="black")
                self.canvas.create_oval(
                    x1 + MARGEN_CIRCULO,
                    y1 + MARGEN_CIRCULO,
                    x2 - MARGEN_CIRCULO,
                    y2 - MARGEN_CIRCULO,
                    fill=color
                )

    def actualizar_estado(self):
        if self.juego_terminado:
            return
        if self.modo == "humano_vs_ia":
            if self.turno == 0:
                self.lbl_estado.config(text="Turno del Humano. Haz clic en una columna.")
            else:
                self.lbl_estado.config(text="Turno de la IA...")
        else:  # ia_vs_ia
            self.lbl_estado.config(text=f"Turno IA {self.turno+1}...")

    def click_humano(self, event):
        if self.juego_terminado:
            return
        if self.modo == "ia_vs_ia":
            # En IA vs IA, no se admiten clics
            return
        if self.turno != 0:
            # Solo el jugador 0 es humano en humano_vs_ia
            return

        col = event.x // TAM_CELDA
        if col < 0 or col >= COLUMNAS:
            return

        if columna_valida(self.tablero, col):
            fila_libre = obtener_fila_libre(self.tablero, col)
            colocar_ficha(self.tablero, fila_libre, col, 1)  # 1 = humano
            self.verificar_estado(1)
            self.turno = 1
            self.actualizar_tablero()
            self.actualizar_estado()
            if not self.juego_terminado:
                # IA juega tras un breve retraso
                self.root.after(500, self.jugada_ia)
        else:
            self.lbl_estado.config(text="Movimiento inválido, elige otra columna.")

    def jugada_ia(self):
        if self.juego_terminado:
            return

        # Determinar qué ficha le corresponde
        ficha = 2 if self.turno == 1 else 1  # En IA vs IA, ambos son IA, 1 = IA1, 2 = IA2
        col = movimiento_ia(self.tablero, self.profundidad, ficha, self.usar_poda)

        if columna_valida(self.tablero, col):
            fila_libre = obtener_fila_libre(self.tablero, col)
            colocar_ficha(self.tablero, fila_libre, col, ficha)
            self.verificar_estado(ficha)
            self.turno = (self.turno + 1) % 2
            self.actualizar_tablero()
            self.actualizar_estado()

            # Si es IA vs IA y el juego no ha terminado, la otra IA juega
            if not self.juego_terminado and self.modo == "ia_vs_ia":
                self.root.after(500, self.jugada_ia)

    def verificar_estado(self, ficha):
        if hay_ganador(self.tablero, ficha):
            self.juego_terminado = True
            if self.modo == "humano_vs_ia":
                if ficha == 1:
                    msg.showinfo("Resultado", "¡Ganaste!")
                    self.lbl_estado.config(text="¡Ganaste!")
                else:
                    msg.showinfo("Resultado", "La IA ha ganado")
                    self.lbl_estado.config(text="La IA ha ganado")
            else:
                msg.showinfo("Resultado", f"¡La IA {ficha} ha ganado!")
                self.lbl_estado.config(text=f"La IA {ficha} ha ganado.")
        elif tablero_lleno(self.tablero):
            self.juego_terminado = True
            msg.showinfo("Resultado", "¡Empate!")
            self.lbl_estado.config(text="¡Empate!")

def main():
    root = tk.Tk()
    app = VentanaConfiguracion(root)
    root.mainloop()

if __name__ == "__main__":
    main()

Referencias:
 - Michie, D. (1966). Game-playing and game-learning automata. In L. Fox (Ed.), Advances in Programming and Non-Numerical Computation (pp. 183-200). Pergamon Press.​
 - Russell, S. J., & Norvig, P. (2010). Artificial Intelligence: A Modern Approach (3rd ed.). Prentice Hall.